LangChain Ollama integration

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

llm = OllamaLLM(model="llama3.1:8b")

chain = prompt | llm

chain.invoke({"question": "What is LangChain?"})

'To break down what LangChain is, let\'s consider the following steps:\n\n1. **Break down the name**: "Lang" is short for "Language", so it suggests that LangChain has something to do with natural language processing (NLP) or human-computer interaction.\n\n2. **Chain implies connection or linking**: The word "chain" implies a sequence of elements or processes connected in some way. This could suggest that LangChain is about connecting different pieces of information, tasks, or even systems together.\n\n3. **Considering the context of technology and AI**: Given the tech-oriented nature of the name, it\'s likely that LangChain has something to do with artificial intelligence (AI) or machine learning (ML). These fields often involve complex connections between data, algorithms, and models.\n\n4. **Linking language and AI/ML**: Combining these insights suggests that LangChain might be an AI-powered tool for handling natural language interactions. This could include tasks like chatbots, tex

LangChain Ollama integration - Chat interface

In [4]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.1:8b")

messages = [
    ("system", "You are a helpful assistant answering user queries."),
    ("human", "What is LangChain?"),
]
llm.invoke(messages)

AIMessage(content="LangChain is an open-source framework for building conversational AI models and applications. It's designed to make it easier to create, integrate, and manage large language models (LLMs) in various use cases.\n\nLangChain provides a set of tools and APIs that enable developers to:\n\n1. **Integrate LLMs** into their applications: LangChain allows you to easily connect popular LLMs like LLaMA, BERT, or T5 to your application.\n2. **Build conversational interfaces**: With LangChain, you can create interactive chatbots, voice assistants, or other conversational interfaces that can understand and respond to user queries.\n3. **Manage and optimize LLM interactions**: The framework provides features for caching, throttling, and monitoring LLM requests to ensure efficient and scalable performance.\n\nLangChain is built on top of popular libraries like PyTorch and TensorFlow, making it compatible with a wide range of development environments.\n\nIts primary goals are to:\n\

Embeddings


In [5]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3.1:8b")

In [10]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [7]:
# embeddings = HuggingFaceEmbeddings(model_name='jinaai/jina-embeddings-v2-base-de')

Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-de and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.10.intermediate.dense.bias', 'encoder.layer.10.intermediate.dense.weight', 'encoder.layer.10.output.LayerNorm.bias', 'encoder.layer.10.output.LayerNorm.weight', 'encoder.layer.10.output.dense.bias', 'encoder.layer.10.output.dense.weight', 'encoder.layer.11.intermediate.dense.bias', 'encoder.layer.11.intermedi

In Memory database

In [11]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

Load PDF Documents

In [12]:
from langchain_community.document_loaders import PyMuPDFLoader

file_path = "./pubhub_docs/Warum Lachen gesund ist_DE.pdf"
loader = PyMuPDFLoader(file_path,
        mode="single",
    )

In [13]:
docs = loader.load()
print(len(docs))
docs[0].metadata

1


{'producer': 'macOS Version 15.3.2 (Build 24D81) Quartz PDFContext',
 'creator': 'Safari',
 'creationdate': "D:20250324210225Z00'00'",
 'source': './pubhub_docs/Warum Lachen gesund ist_DE.pdf',
 'file_path': './pubhub_docs/Warum Lachen gesund ist_DE.pdf',
 'total_pages': 4,
 'format': 'PDF 1.4',
 'title': 'Warum Lachen gesund ist',
 'author': 'David Massimo',
 'subject': '',
 'keywords': '',
 'moddate': "D:20250324210225Z00'00'",
 'trapped': '',
 'modDate': "D:20250324210225Z00'00'",
 'creationDate': "D:20250324210225Z00'00'"}

In [14]:
#
# Loading + Parsing files
#

from langchain_community.document_loaders import FileSystemBlobLoader
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import PyMuPDFParser

loader = GenericLoader(
    blob_loader=FileSystemBlobLoader(
        path="./pubhub_docs/",
        glob="*.pdf",
    ),
    # blob_parser=PyMuPDFParser(mode="single"),
    blob_parser=PyMuPDFParser(mode="single"),
)
docs = loader.load()
len(docs[5].page_content)
# docs[0].metadata

9119

Tokenization - Doc splitting

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 68 sub-documents.


Store splits

In [16]:
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

['ee8affaa-657a-49bb-97c4-b3305e3dcca4', '2b640e6b-2d5e-4955-81c1-a7d8ce16bde0', 'f8cb7bcb-6017-4c76-9f50-e68972fe3868']


Retrieval and Generation

In [17]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "(context goes here)", "question": "(question goes here)"}
).to_messages()

assert len(example_messages) == 1
print(example_messages[0].content)

/Users/david/miniconda3/envs/hf_gpu/lib/python3.11/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: (question goes here) 
Context: (context goes here) 
Answer:


In [18]:
from langchain_core.prompts import PromptTemplate

template = """You are an helpful chatbot to support workers at Public Value Technologies gmbh (also known as pub.tech). 
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Always say "thanks for asking!" at the end of the answer.

If the query is in German you reply in german, otherwise in english.
{context}

Question: {question}

Helpful Answer:"""
prompt = PromptTemplate.from_template(template)

In [37]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict

from pydantic import BaseModel, Field

class CitedAnswer(BaseModel):
    """Answer the user question based only on the given sources, and cite the sources used."""

    answer: str = Field(
        ...,
        description="The answer to the user question, which is based only on the given sources.",
    )
    citations: List[int] = Field(
        ...,
        description="The integer IDs of the SPECIFIC sources which justify the answer.",
    )

def format_docs_with_id(docs: List[Document]) -> str:
    formatted = [
        f"Source ID: {i}\nArticle Title: {doc.metadata['source']}\nArticle Snippet: {doc.page_content}"
        for i, doc in enumerate(docs)
    ]
    return "\n\n" + "\n\n".join(formatted)



class State(TypedDict):
    question: str
    context: List[Document]
    # answer: str
    answer: CitedAnswer

# Nodes
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"],k=3)
    return {"context": retrieved_docs}


# def generate(state: State):    
#     docs_content = "\n\n".join(doc.page_content for doc in state["context"])
#     messages = prompt.invoke({"question": state["question"], "context": docs_content})
#     # print(messages)
#     response = llm.invoke(messages)
    # return {"answer": response}

def generate(state: State):
    formatted_docs = format_docs_with_id(state["context"])
    messages = prompt.invoke({"question": state["question"], "context": formatted_docs})
    structured_llm = llm.with_structured_output(CitedAnswer)
    response = structured_llm.invoke(messages)
    return {"answer": response}


# Control flow of the program (langgrap)h
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [38]:
import pprint

[1, 2]

In [47]:
# result = graph.invoke({"question": "Was soll ich tun um urlaub zu buchen at pub.tech?"})
result = graph.invoke({"question": "What should I do when doing a businesss trip?"})

pprint.pp(f'Context: {result["context"]}\n\n')
pprint.pp(f'Answer: {result["answer"]}')

for cit in result["answer"].citations:
    print(result["context"][cit].metadata["source"])


("Context: [Document(id='96471a83-58fe-446d-b81b-1440a229657f', "
 "metadata={'producer': 'macOS Version 15.3.2 (Build 24D81) Quartz "
 "PDFContext', 'creator': 'Safari', 'creationdate': "
 '"D:20250324210537Z00\'00\'", \'source\': \'pubhub_docs/Business trips and '
 "travel expense guidelines.pdf', 'file_path': 'pubhub_docs/Business trips and "
 "travel expense guidelines.pdf', 'total_pages': 3, 'format': 'PDF 1.3', "
 "'title': 'Business trips and travel expense guidelines', 'author': 'David "
 "Massimo', 'subject': '', 'keywords': '', 'moddate': "
 '"D:20250324210537Z00\'00\'", \'trapped\': \'\', \'modDate\': '
 '"D:20250324210537Z00\'00\'", \'creationDate\': "D:20250324210537Z00\'00\'", '
 "'start_index': 2460}, page_content='24/03/25, 22:05\\nBusiness trips and "
 'travel expense guidelines\\nPage 2 of '
 '3\\nhttps://publicvaluetech.sharepoint.com/sites/for-you/SitePages/en/Dienstreisen-und-Reisekostenrichtlinien.aspx\\nThe '
 'balance between meeting business requirements, compl

In [50]:
result["answer"].answer

"Please refer to the Business Trips and Travel Expense Guidelines document (pubhub_docs/Business trips and travel expense guidelines.pdf) on our SharePoint site. This will provide you with detailed information on what to do when planning a business trip, including regulations on working hours, rest breaks, and reimbursement of expenses. Additionally, it's essential to ensure that you are adequately insured during the trip. If you're unsure about any aspect of a business trip, please consult your supervisor or HR department."

In [45]:
result["context"][1].metadata["source"]

'pubhub_docs/Business trips and travel expense guidelines.pdf'

In [26]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
model_name = 'qanastek/51-languages-classifier'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer)
res = classifier("was soll ich machen")
print(res)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'de-DE', 'score': 0.9999765157699585}]


In [27]:
from pydantic import BaseModel, Field


class CitedAnswer(BaseModel):
    """Answer the user question based only on the given sources, and cite the sources used."""

    answer: str = Field(
        ...,
        description="The answer to the user question, which is based only on the given sources.",
    )
    citations: List[int] = Field(
        ...,
        description="The integer IDs of the SPECIFIC sources which justify the answer.",
    )

def format_docs_with_id(docs: List[Document]) -> str:
    formatted = [
        f"Source ID: {i}\nArticle Title: {doc.metadata['source']}\nArticle Snippet: {doc.page_content}"
        for i, doc in enumerate(docs)
    ]
    return "\n\n" + "\n\n".join(formatted)



structured_llm = llm.with_structured_output(CitedAnswer)

example_q = """What Brian's height?

Source: 1
Information: Suzy is 6'2"

Source: 2
Information: Jeremiah is blonde

Source: 3
Information: Brian is 3 inches shorter than Suzy"""
result = structured_llm.invoke(example_q)

result

CitedAnswer(answer='5\'11"', citations=[3])

In [34]:
docs[0].metadata["source"]

'pubhub_docs/FAQ - Workation_DE.pdf'